In [2]:
import numpy as np
from tqdm import tqdm

# Load the uploaded file
file_path = 'msnbc990928.seq'  # change if needed

# Read sessions from file
with open(file_path, 'r') as f:
    sessions = []
    for line in f:
        line = line.strip()
        if not line or line.startswith('%'):
            continue  # skip empty and comment lines
        try:
            sessions.append(list(map(int, line.split())))
        except ValueError:
            print("Skipping invalid line:", line)


print(f"Total sessions loaded: {len(sessions)}")

# Filter sessions with 3–6 page visits (as per the paper)
sessions = [s for s in sessions if 3 <= len(s) <= 6]
print(f"Filtered sessions (3–6 pages): {len(sessions)}")

# Simulate time blocks (e.g., 10 blocks = 10 time periods)
n_blocks = 10
block_size = len(sessions) // n_blocks
blocks = [sessions[i * block_size:(i + 1) * block_size] for i in range(n_blocks)]

n_pages = 17  # MSNBC dataset has 17 page categories

# Build transition matrices
transition_matrices = []
for block in blocks:
    matrix = np.zeros((n_pages, n_pages))
    for session in block:
        for i in range(len(session) - 1):
            from_page = session[i] - 1
            to_page = session[i + 1] - 1
            matrix[from_page][to_page] += 1
    # Normalize rows
    with np.errstate(divide='ignore', invalid='ignore'):
        row_sums = matrix.sum(axis=1, keepdims=True)
        matrix = np.divide(matrix, row_sums, out=np.zeros_like(matrix), where=row_sums != 0)
    transition_matrices.append(matrix)

# Compute Q_SESA using exponential smoothing
mu = 0.25
Q_sesa = sum(mu * ((1 - mu) ** i) * Q for i, Q in enumerate(reversed(transition_matrices)))

# Define the link prediction function (LPSESA)
def predict_next_page(session, Q, weights):
    n = Q.shape[0]
    N = np.zeros(n)
    for k, w in enumerate(weights):
        if len(session) <= k:
            continue
        page = session[-1 - k] - 1
        L = np.zeros(n)
        L[page] = 1
        Q_pow = np.linalg.matrix_power(Q, k + 1)
        N += w * (L @ Q_pow)
    return np.argmax(N) + 1

# Evaluate model accuracy
weights = [0.9, 0.5, 0.3, 0.2, 0.1]
correct = 0

for s in tqdm(sessions):
    hist, actual = s[:-1], s[-1]
    pred = predict_next_page(hist, Q_sesa, weights[:len(hist)])
    if pred == actual:
        correct += 1

accuracy = correct / len(sessions)
print(f"\n✅ LPSESA Prediction Accuracy: {accuracy:.4f}")


Skipping invalid line: frontpage news tech local opinion on-air misc weather msn-news health living business msn-sports sports summary bbs travel
Total sessions loaded: 989818
Filtered sessions (3–6 pages): 267972


100%|██████████| 267972/267972 [00:09<00:00, 29202.95it/s]


✅ LPSESA Prediction Accuracy: 0.6610


In [3]:
import random

# --- Evaluate Q_sesa + prediction accuracy ---
def compute_Q_sesa(mu, transition_matrices):
    return sum(mu * ((1 - mu) ** i) * Q for i, Q in enumerate(reversed(transition_matrices)))

def evaluate_accuracy(Q_sesa, sessions, weights):
    correct = 0
    for s in sessions:
        if len(s) < 2:
            continue
        history, actual = s[:-1], s[-1]
        pred = predict_next_page(history, Q_sesa, weights[:len(history)])
        if pred == actual:
            correct += 1
    return correct / len(sessions)

# --- GA Parameters ---
population_size = 50
generations = 30
mutation_rate = 0.4
elite_size = 10

# --- Mixed mutation: fine + coarse steps ---
def mutate(mu):
    if random.random() < mutation_rate:
        step = random.choice([0.05, 0.1, 0.2])
        direction = random.choice([-1, 1])
        mu += direction * step * random.uniform(0.5, 1.0)
        mu = max(0.01, min(mu, 0.99))
    return mu

# --- Initialize population ---
population = [random.uniform(0.01, 0.99) for _ in range(population_size)]
global_best_mu = None
global_best_acc = 0

# --- GA loop ---
for gen in range(generations):
    fitness_scores = []
    
    for mu in population:
        Q = compute_Q_sesa(mu, transition_matrices)
        acc = evaluate_accuracy(Q, sessions, weights)
        fitness_scores.append((mu, acc))

    # Sort and update global best
    fitness_scores.sort(key=lambda x: x[1], reverse=True)
    best_mu, best_acc = fitness_scores[0]

    if best_acc > global_best_acc:
        global_best_acc = best_acc
        global_best_mu = best_mu

    print(f"Generation {gen+1}/{generations} → Best μ: {best_mu:.4f} | Acc: {best_acc:.4f}")

    # Elitism: carry over top performers
    new_population = [mu for mu, _ in fitness_scores[:elite_size]]

    # Generate rest via crossover + mutation
    while len(new_population) < population_size:
        p1, p2 = random.sample(fitness_scores[:20], 2)
        child_mu = (p1[0] + p2[0]) / 2
        child_mu = mutate(child_mu)
        new_population.append(child_mu)

    population = new_population

# --- Final results ---
print(f"\n🎯 Optimal μ found: {global_best_mu:.4f}")
Q_opt = compute_Q_sesa(global_best_mu, transition_matrices)
final_acc = evaluate_accuracy(Q_opt, sessions, weights)
print(f"✅ Final LPSESA Accuracy with Optimal μ: {final_acc:.4f}")


Generation 1/30 → Best μ: 0.1577 | Acc: 0.6638
Generation 2/30 → Best μ: 0.1577 | Acc: 0.6638
Generation 3/30 → Best μ: 0.1577 | Acc: 0.6638


KeyboardInterrupt: 